In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
# Read in all the csv files from the data folder and save them as a dictionary of DataFrames
data_dir = Path('data/data')
dfs = {}

try:
    for file_path in data_dir.glob('*.csv'):
        if file_path.is_file():
            # Get the filename without extension
            file_name = file_path.stem  
            df = pd.read_csv(file_path)
            dfs[file_name] = df
except Exception as e:
    print(f"Error occurred: {e}")

# Check the keys of the dictionary
print(dfs.keys())

# Save each DataFrame as individual dfs
for key, df in dfs.items():
    globals()[f'df_{key}'] = df

dict_keys(['comments', 'follows', 'likes', 'photos', 'photo_tags', 'tags', 'users'])


In [3]:
# Check for nulls in each df
for key, df in dfs.items():
    print(f"Nulls in DataFrame {key}:")
    print(df.isnull().sum().sum())

Nulls in DataFrame comments:
0
Nulls in DataFrame follows:
0
Nulls in DataFrame likes:
0
Nulls in DataFrame photos:
0
Nulls in DataFrame photo_tags:
0
Nulls in DataFrame tags:
0
Nulls in DataFrame users:
0


In [23]:
# Replace spaces with underscores in the column names and make all lowercase
for key, df in dfs.items():
    df.columns = df.columns.str.replace(' ', '_')
    df.columns = df.columns.str.lower()

In [25]:
# Change yes/no columns to boolean values and data type
for key, df in dfs.items():
    for col in df.columns:
        # If columns only contain yes or no, change to boolean
        if df[col].isin(['yes', 'no']).all():
            df[col] = df[col].map({'yes': True, 'no': False})

In [27]:
from datetime import datetime, timedelta
# Convert the created columns to datetime data type
date_time_cols = ['created_dat','created_date', 'created_time', 'created_timestamp']

for key, df in dfs.items():
    for col in date_time_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')  # Coerce will return NaT for invalid dates

In [28]:
df.head()

,user_id,name,created_time,private,post_count,verified_status
0,1,Kenton_Kirlin,2017-02-16 18:22:00,True,11,False
1,2,Andre_Purdy85,2017-04-02 17:11:00,False,7,False
2,3,Harley_Lind18,2017-02-21 11:12:00,False,2,False
3,4,Arely_Bogan63,2016-08-13 01:28:00,True,1,False
4,5,Aniya_Hackett,2016-12-07 01:04:00,True,3,False


In [30]:
# Create new id columns for the likes and follows dataframes as they will be used as foreign keys in the interactions table
df_likes['like_id'] = (df_likes.index + 1).astype(int)
df_follows['follow_id'] = df_follows.index + 1

In [31]:
# Rename the columns in each dataframe to match the database schema
df_photos.rename(columns={'created_dat': 'created_date','id':'photo_id'}, inplace=True)
df_users.rename(columns={'id': 'user_id','private/public':'private'}, inplace=True)
df_tags.rename(columns={'id': 'tag_id'}, inplace=True)
df_comments.rename(columns={'user__id': 'user_id','id':'comment_id','created_timestamp':'created_time'}, inplace=True)
df_likes.rename(columns={'user_': 'user_id', 'photo': 'photo_id'}, inplace=True)
df_follows.rename(columns={'follower': 'follower_user_id', 'followee_': 'user_id'}, inplace=True)

In [32]:
# Combine Likes, Comments, and Follows DataFrames into a single Interactions DataFrame
df_interactions = pd.concat([df_likes, df_comments, df_follows], ignore_index=True)

# Create interaction_id column
df_interactions['interaction_id'] = df_interactions.index + 1

# Add a new column to indicate the type of interaction ('like', 'comment', or 'follow')
df_interactions['interaction_type'] = pd.Series(['like'] * len(df_likes) + ['comment'] * len(df_comments) + ['follow'] * len(df_follows))

# Add an interaction date column (you can use 'created_time' column or any other appropriate date column)
df_interactions['interaction_date'] = df_interactions['created_time']

# List of columns to keep
keep_cols = ['interaction_id', 'interaction_type', 'interaction_date', 'user_id', 'photo_id', 'comment_id', 'like_id','follow_id']

# Remove columns that are not in the keep_cols list
df_interactions = df_interactions[[col for col in df_interactions.columns if col in keep_cols]]

# Make sure the id columns are integers even though they contain NaN values
id_cols = ['user_id', 'photo_id', 'comment_id', 'like_id', 'follow_id']
for col in id_cols:
    df_interactions[col] = df_interactions[col].astype('Int64')

In [33]:
df_interactions.head()

,user_id,photo_id,like_id,comment_id,follow_id,interaction_id,interaction_type,interaction_date
0,2,1,1,<NA>,<NA>,1,like,2023-04-13 08:04:00
1,2,4,2,<NA>,<NA>,2,like,2023-04-13 08:04:00
2,2,8,3,<NA>,<NA>,3,like,2023-04-13 08:04:00
3,2,9,4,<NA>,<NA>,4,like,2023-04-13 08:04:00
4,2,10,5,<NA>,<NA>,5,like,2023-04-13 08:04:00
